In [117]:
import pandas as pd
from scipy.spatial.distance import *
import pickle 
import random
from time import sleep
import json
import requests
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [2]:
#отключение предупреждений
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
groups = pd.read_csv('ekb_community_stats.csv', header=None).values.ravel().tolist()

Для анализа было решено использовать часть данных, так как доступные мне вычислительные мощности не позволяют обрабатывать все предоставленные данные. Подвыборка выбиралась случайно.

In [ ]:
subsample_size = 20000
rand_groups = []
for i in range(subsample_size):
    group = random.choice(groups)
    rand_groups.append(group)
    groups.remove(group)  

In [ ]:
pickle.dump(rand_groups, open( "rand_groups_origin", "wb" ))
wall_get()

In [ ]:

def wall_get(id):
    """
    Keyword arguments:
    id - group's id
    
    returns [0] if error
    otherwise reterns [number of posts in group, last 100 posts]
    """
    first = "https://api.vk.com/method/wall.get?owner_id=-"
    last = "&count=100"
    data = requests.get(first + str(id) + last)
    datas = data.content.decode('utf8')
    data = json.loads(datas)
    try:
        data = data['response']
    except:
        return [0]
    return data[0], data[1:]

# Загрузка данных

В некоторых группах из выборки контент представлен в виде картинок. Было решено разделить группы, где информация передается с помощью картинок и где информация передается с помощью текста. Часть групп из воборки представляет собой закрытые, заблокированные сообщества, а так же сообщества, в которых недостаточно информации для анализа (мало постов). Такие группы удалялись. 

* av_word_len - средняя длина слова
* pic_groups_list - список идентификаторов групп с картинками
* text_groups_list список идентификаторов групп с текстом
* deleted_groups - список идентификаторов удаленных групп
* text_groups_dict словарь, содержащий список из постов групп. Ключем является идентефикатор группы

In [ ]:
"""av_word_len = 7
pic_groups_list = []
text_groups_list = []
text_groups_dict = {}
deleted_groups = []

j = 0"""

for id in rand_groups:

    posts = wall_get(id)
    
    
    if posts[0] <= 15: #если мало постов, удаляем группу
        j += 1
        deleted_groups.append(id)
        continue
      
    if posts[0] < 100: 
        n_posts = posts[0]
    else:
        n_posts = 100
    group_posts = []
    group_len = 0
    
    for post in posts[1]:
        new_post = ' '.join(post['text'].split('<br>'))
        group_len += len(new_post)
        group_posts.append(new_post)
    
    if float(group_len) / (n_posts * av_word_len) < 6.5: #Если количество слов на пост в группе < 6.5, добавляем в pic
        pic_groups_list.append(id)
    else:
        text_groups_dict[str(id)] = group_posts
        text_groups_list.append(id)
    
    j += 1
    print(j, id)


In [149]:
"""Во время загрузки данных у меня отключился интернет, 
которого не было в последствии весь день, поэтому для 
анализа использовалось меньше групп, чем указано выше."""

len(text_groups_list), len(pic_groups_list), len(deleted_groups)

(8082, 1474, 3434)

In [ ]:
#Сохранение данных
pickle.dump(deleted_groups, open( "deleted_groups", "wb" ))
pickle.dump(pic_groups_list, open( "pic_groups_list", "wb" ))
pickle.dump(text_groups_list, open( "text_groups_list", "wb" ))
pickle.dump(text_groups_dict, open( "text_groups_dict", "wb" ))

# Кластеризация групп с текстом

Для кластеризации групп с текстом было решено использовать алгоритм K-means. Все посты были лемматизированы. После, все посты из одной группы были склеены в один текст. После данные были преобразованы с помощью алгоритма tf-idf.

In [ ]:
#Лемматизация
m = Mystem()
keys = list(text_groups_dict.keys())
j = 0

for key in keys:
    for i in range(len(text_groups_dict[key])):
        text_groups_dict[key][i] = ''.join(m.lemmatize(text_groups_dict[key][i]))
    j += 1
    print(j, key)

In [ ]:
#Сохранение данных
pickle.dump(text_groups_dict, open( "text_groups_dict_lem", "wb" ))

In [ ]:
#Склейка всех постов в каждой группе
text_group_dict2 = {}
for key in keys:
    text_group_dict2[key] = ''.join((''.join(text_groups_dict[key])).split("\n"))       
        

In [ ]:
#Сохранение данных
pickle.dump(text_group_dict2, open( "text_group_dict_lem2", "wb" ))

In [3]:
#Загрузка данных
data_dict = pickle.load(open('text_group_dict_lem2', 'rb'))
data = list(data_dict.values())

In [4]:
#Загрузка стоп слов
file = open("stop_words.txt", encoding='utf-8')
stop_words = file.read().split("\n")
file.close()

In [144]:
vectorizer = TfidfVectorizer(
            max_df=0.5,
            max_features=5000,
            min_df=2,
            stop_words=stop_words,
            analyzer="word")

In [ ]:
#Преобразование данных с помощью tf-idf
X = vectorizer.fit_transform(data)
X = X.toarray()

Ниже можно увидеть отобранные tf-idf слова.

In [147]:
for i in vectorizer.get_feature_names():
    print(i)

adidas
air
albums
alex
ali
aliexpress
android
app
apple
arena
art
article
articles
aspx
auto
bar
bass
bb
beach
beauty
best
big
bit
biz
black
blog
blue
bmw
buy
canon
catalog
city
club
co
concert
core
ctrl
dance
day
de
deep
deluxe
design
digital
dj
dlya
dress
drum
ekb
english
ep
event
events
facebook
fashion
feat
fest
festival
fi
fm
forum
free
ft
full
galaxy
get
girl
gl
gmail
go
gold
goo
good
google
grand
group
hair
hall
happy
hardcore
hd
hip
hiphop
home
hop
hotel
house
id
ii
iii
index
info
ios
ip
ipad
iphone
item
itunes
kak
kit
la
label
led
lg
life
like
line
live
look
love
lt
ly
mail
make
man
market
mercedes
mini
mix
model
money
moscow
music
na
net
new
night
nike
novosti
one
online
open
org
original
page
park
party
photo
photographer
php
play
plus
post
potato
power
pr
pro
product
project
pub
qiwi
radio
rap
records
red
release
remix
resort
rock
rub
russia
sale
samsung
seo
shop
show
skype
sms
sony
soundcloud
spa
space
spb
sport
star
start
steam
store
story
studio
style
su
summer
tattoo
te

In [111]:
n_clusters = 20
km = KMeans(n_clusters=n_clusters, init='k-means++', n_init=1)
                         

In [112]:
km.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=20, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [113]:
terms = vectorizer.get_feature_names()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(n_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: руб платье наличие размер заказ цвет доставка скидка ткань см
Cluster 1: iphone игра apple сервер смартфон samsung приложение android устройство galaxy
Cluster 2: твой любовь мужчина женщина душа сердце счастие никто глаз рядом
Cluster 3: секс парень мужчина искать женщина анон член анонимно собака познакомиться
Cluster 4: бизнес сайт зарабатывать доход компания интернет деньги рубль клиент проект
Cluster 5: добавлять квартира сайт рубль видео конкурс руб тело привет игра
Cluster 6: масло ложка сахар яйцо соль добавлять тесто ингредиент мука перец
Cluster 7: remix mix original house feat dj bass drum techno trance
Cluster 8: екатеринбург конкурс фестиваль программа проект занятие участие ул язык участник
Cluster 9: упражнение женщина мышца книга энергия мужчина сила отношение организм вода
Cluster 10: заказ доставка заказывать скидка наличие товар рубль подарок topic размер
Cluster 11: волос кожа наращивание ресница процедура бровь маска масло макияж ноготь
Cluster 12: свадь

Как можно заметить, k-means довольно неплохо справляется с кластеризацией текста. Кластеры, полученные с помощью такого подхода вполне можно назвать тематическими. Тем не менее, сложность заключается в выборе кластеров. Поставив слишком большое количество кластеров, мы можем получить повторяющиеся кластеры, а поставив маленькое, можем не получить важной информации.

# Иерархическая кластеризация с помощью k-means

Для того, чтобы устранить недостаток, описанный выше, было решено запустить k-means делить данные рекурсивно по два кластера. Таким образом можно построить дерево, в узлах которой будут данные о конкретных кластерах. Критерием остановки будет служить максимальный размер кластера (max_cl_size). Если кластер меньше максимального размера кластера, то происходит выход из рекурсии.

Данные о кластерах:

* data - векторы, полученные из матрицы X
* level - глубина дерева, на которой лежит кластер
* size - размер кластера
* words - ключевые слова кластера
* groupd - идентефикаторы групп кластера
* center - центр кластера

Позже, для каждого кластера будет добавлено еще одно поле:

* ind - индекс кластера на определенной глубине


In [ ]:
def get_clust_words(k_means, ind, num=10):
    """
    Keyword arguments:
    k_means - k-means object
    ind - cluster's index
    num - numbers of words to return
    
    returns num ind-th cluster's key words
    """
    terms = vectorizer.get_feature_names()
    order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]
    
    words = []
    for i in order_centroids[ind, :num]:
        words.append(terms[i])
    return words


def recurent_km(data, groups, cluster_words=None, center=None, max_cl_size=400, k=1, stage=0):    
    global max_level
    if stage == 0:
        max_level = 0
    if max_level < stage:
        max_level = stage
    clust_dict = {}
    clust_dict['data'] = data
    clust_dict['level'] = stage
    clust_dict['size'] = len(data)
    clust_dict['words'] = cluster_words
    clust_dict['groups'] = groups
    clust_dict['center'] = center
    
    
    if len(data) < max_cl_size:
        list_tree[k-1] = clust_dict
        return
    
    list_tree[k-1] = clust_dict
    
    km = KMeans(n_clusters=2, init='k-means++', n_init=1)
    
    km.fit(data) 
    data1 = []
    data2 = []
    groups1 = []
    groups2 = []
    center1 = km.cluster_centers_[0]
    center2 = km.cluster_centers_[1]
    cluster_words1 = get_clust_words(km, 0)
    cluster_words2 = get_clust_words(km, 1)
    
    labels = km.labels_
    for i in range(len(labels)):
        if labels[i] == 0:
            data1.append(data[i])
            groups1.append(groups[i])
        else:
            data2.append(data[i])
            groups2.append(groups[i])
    recurent_km(data=data1, groups=groups1, cluster_words=cluster_words1, center=center1, k=2*k, stage=stage+1)
    recurent_km(data=data2, groups=groups2, cluster_words=cluster_words2, center=center2, k=2*k + 1, stage=stage+1)

list_num = 2**23
list_tree = [0] * list_num      #лист, куда будут записываться кластера (взят с запасом)
max_cl_size = 400               #максимальный размер кластера
groups = list(data_dict.keys()) # идентефикаторы групп
recurent_km(data=X, groups=groups)

In [253]:
print("Глубина дерева:", max_level)

Глубина дерева: 15


In [72]:
#Разделение на уровни и присвоение каждому кластеру индекса

levels = []

for i in range(max_level + 2):
    levels.append(list_tree[(2**i)-1: 2**(i+1) - 1])

for level in levels[1:]:
    for i in range(len(level)):
        if level[i] != 0:
            level[i]['ind'] = level.index(level[i])

In [64]:
#Нахождение конечных кластеров
down_cl = []
for level in levels[1:]:
    for cluster in level:
        if cluster != 0:
            if cluster['size'] < max_cl_size:
                down_cl.append(cluster)

Получив дерево кластеров, можно посмотреть на конечные кластеры (кластеры, размер которах не превышает max_cl_size) и решить, удовлетворяют ли эти кластерам заданным требованиям. Если конечные кластеры можно разделить на подклассы, то следует запустить алгоритм, уменьшив max_cl_size. Если конечные кластеры повторяют друг друга, можно, с помощью вспомогательной функции получить их родителя.

In [105]:
def get_ancestor(cluster):
    """
    Keyword arguments:
    cluster - cluster's data from list_tree
    
    returns cluster's ancestor
    """
    level = levels[cluster['level'] - 1]
    ind = cluster['ind'] // 2
    
    return level[ind] 

def sort_groups(cluster):
     """
    Keyword arguments:
    cluster - cluster's data from list_tree
    
    returns sorted by the distance frome the center groups
    """
    center = cluster['center']
    groups = cluster['groups']
    data = cluster['data']
    dist_matrix = []
    sorted_groups = []
    for vec in data:
        dist_matrix.append(euclidean(vec, center))
    sorted_ind = np.array(dist_matrix).argsort()
    for ind in sorted_ind:
        sorted_groups.append(groups[ind])
    return sorted_groups
    

Ниже можно увидеть для каждого кластера его размер, глубину на которой он находится, его индекс и ключевые слова

In [255]:
print("Количество коечных кластеров:",len(down_cl), '\n')

for i in down_cl:
    print(i['size'], i['level'], i['ind'], *i['words'])

Количество коечных кластеров: 42 

395 2 1 свадьба свадебный невеста фотограф фотосессия съемка wedding прическа макияж платье
7 3 4 утро слабо осень ночь сердце твой верить музыка спокойный любовь
204 3 6 книга фильм бог история земля любовь женщина смерть жанр сила
205 4 3 масло мышца сахар упражнение яйцо ложка тренировка соль добавлять ингредиент
15 4 10 бог божий господь любовь молитва преподобный иоанн твой христос святой
138 4 14 малыш вода масло мама ложка организм кожа упражнение сахар добавлять
330 4 15 деньги женщина мужчина бизнес отношение цель энергия книга любовь проблема
142 5 0 секс искать парень познакомиться член вирт анон привет встреча куни
251 5 2 руб iphone чехол apple samsung продавать рубль размер смартфон наличие
140 6 6 платье наличие размер заказ цвет руб ткань одежда юбка скидка
383 6 9 квартира сайт компания интернет рубль доход зарабатывать бизнес руб проект
267 6 10 концерт билет альбом песня музыка фестиваль клуб рок екатеринбург москва
149 6 11 альбом 

Как можно заметить, следующие кластера очень похожи между собой, поэтому мы можем посмотреть на их предка

* 240 6 44 твой любовь душа сердце никто скучать рядом счастие прощать уходить"
* 239 6 45 твой любовь мужчина женщина сердце счастие душа счастливый глаз никто

In [101]:
print(*get_ancestor(levels[6][44])['words'])
print(*get_ancestor(levels[6][45])['words'])

твой любовь сердце душа мужчина счастие женщина никто рядом счастливый
твой любовь сердце душа мужчина счастие женщина никто рядом счастливый


Ниже предоставлены ключевые слова по каждому кластеру и по 3 группы, наиболее приближенные к центру кластера


In [110]:
for cluster in down_cl:
    groups = sort_groups(cluster)
    print(*cluster['words'])
    print('\nvk.com/club' + groups[0])
    print('vk.com/club' + groups[1])
    print('vk.com/club' + groups[2], '\n')
    

свадьба свадебный невеста фотограф фотосессия съемка wedding прическа макияж платье

vk.com/club32768467
vk.com/club24251666
vk.com/club34752316 

утро слабо осень ночь сердце твой верить музыка спокойный любовь

vk.com/club30790069
vk.com/club46837498
vk.com/club45192348 

книга фильм бог история земля любовь женщина смерть жанр сила

vk.com/club51682933
vk.com/club45270133
vk.com/club18206367 

масло мышца сахар упражнение яйцо ложка тренировка соль добавлять ингредиент

vk.com/club54725864
vk.com/club48946342
vk.com/club32455286 

бог божий господь любовь молитва преподобный иоанн твой христос святой

vk.com/club16838988
vk.com/club29599237
vk.com/club36825554 

малыш вода масло мама ложка организм кожа упражнение сахар добавлять

vk.com/club82274301
vk.com/club54616802
vk.com/club42082503 

деньги женщина мужчина бизнес отношение цель энергия книга любовь проблема

vk.com/club54087668
vk.com/club47211280
vk.com/club51563246 

секс искать парень познакомиться член вирт анон привет в

# Кластеризация групп с картинками

Изначально, мной была допущена ошибка, из-за которой лист pic_groups_list записались группы, содержащие музыку. Ниже группы из pic_groups_list были разделены на music (группы с музыкой) и pic (группы с картинками).

Первый кластер (music) можно объеденить с аналогичным кластеров (кластер, ключевыми словами которого являются музыкальные жанры). Второй же (pic) было решено оставить, так как анализ картинок - довольно трудоемкая задача, которая, в контексте данного задания, не принесет большой пользы.

In [245]:
def get_wall2(id):
    first = "https://api.vk.com/method/wall.get?owner_id=-"
    last = "&count=100"
    data = requests.get(first + str(id) + last)
    datas = data.content.decode('utf8')
    data = json.loads(datas)
    try:
        data = data['response']
    except:
        data = 0
    return data

In [ ]:
pic = []
music = []
bad_id = []
j = 0
for id in pic_groups_list:
    wall = get_wall2(id)
    audio = 0
    if wall == 0:
        bad_id.append(id)
    else:
        for i in range(2, len(wall) - 2):
            try:
                attachments =  wall[i]['attachments']
                for attach in attachments:
                    try:
                        _ = attach['audio']
                        audio += 1
                    except:
                        continue 
            except:
                continue
                
        if audio/(len(wall) - 2) < 1:
            pic.append(id)
        else:
            music.append(id)

In [256]:
print(len(music), len(pic))

109 1364
